In [1]:
import os;os.environ['HIVESERVER2_HOST'] = 'bbx.farlightgames.com'
import sys;sys.path.append('../../')

import pendulum
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, trange
from cityhash import CityHash32


from config.HIVESERVER2_token import *
from pyutil.pysql.sql import SQLClient

from tfutils.layers.attn import BatchNormTransformerEncoder
from tfutils.layers.vsn import FieldVariableSelectionWithContext


dense_vocab = {
    'r_hit_number' : [0,37,52,65,79,94,110,131,160,204],
    'r_hited_number' : [0,25,34,41,47,54,62,71,84,108],
    'r_headshot_number' : [0,1,3,4,6,7,9,11,14,19],
    'r_headshoted_number' : [0,1,2,2,3,4,5,7,9],
    'r_damage_amount' : [0,444,619,768,930,1105,1316,1578,1954,2556], 
    'r_damage_taken_amount' : [0,394,497,570,641,697,770,862,984,1169],
    'r_kill_number' : [0,1,2,3,4,5,6,8,11], 
    'r_assist_number' : [0,1,2,3,4],
    'r_dead_number' : [0,1,2,3], 
    'r_death_number' : [0,1,2,3],
    'r_ka_per_3min' : [0.053333065290828305,0.5819396952264781,0.8707765252887384,1.132015890606052,1.3997278853044248,1.677375474831049,1.9912202837942263,2.379640123420753,2.9378460382063785], 
    'r_survival_time' : [0, 392,496,555,618,691,764,852,929,1008],
    'r_hit_rate' : [0.14172694809289044,0.1962671555074827,0.23181147113567696,0.2628059253769059,0.2925804733319646,0.3254649310700026,0.362230869508433,0.4163552085422169,0.5181101700409702],
    'r_headshot_rate' : [3.11899816415794E-5,0.005770877578211221,0.010592320691133663,0.014985647680509492,0.019293363869190706,0.024109606455985037,0.030168272799344437,0.03820006451244755,0.05322657625320898],

    'r_save_number' : [0, 1, 2, 3, 4],
    'r_fade_time' : [0,1,2,4,8,16,24,36,48],
    'r_rank_team' : [-4.,0.,3.,8.,16.,24.,30.,40.],
    'r_safearea_rank' : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],

    'r_elo' : list(range(0, 4200, 300)),
    'r_his_battle_number_3d' : [1,2,4,8,16],
    'r_lifespan' : [0, 3, 7, 14, 30, 60,120, 180],
}

cat_vocab = {
    'r_weapon1' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_weapon2' : [11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, 21001, 22001, 23001, 24001, 25001, 26001, 27001, 28001, 29001, 30001, 32001],
    'r_character_id' : [100002, 100003, 100004, 100005, 100006, 100007, 100008, 100009, 100010, 100011, 100012, 100013, 100014, 100015, 100016, 100017, 100018, 100019, 100020, 100021, 100022, 100023, 100024, 100025, 100026, 100027],
}

vocab = dict(
    dense_vocab = dense_vocab,
    cat_vocab = cat_vocab
)

inputs = {
    k : tf.keras.layers.Input((5,), dtype=tf.float32, name=k)
    if k not in ['r_elo', 'r_lifespan', 'r_his_battle_number_3d']
    else tf.keras.layers.Input((1,), dtype=tf.int32, name=k)
    for k, v in dense_vocab.items()
}

inputs.update({
    k : tf.keras.layers.Input((5,), dtype=tf.int32, name=k)
    for k, v in cat_vocab.items()
})

def build_encoder(inputs, hidden_size : int = 32):
    embeddings = {}
    
    for k in dense_vocab.keys():
        x = inputs[k]
        x = tf.keras.layers.Discretization(dense_vocab[k])(x)
        x = tf.keras.layers.Embedding(len(dense_vocab[k]) + 1, hidden_size)(x)
        embeddings[k] = x
        
    for k in cat_vocab.keys():
        x = inputs[k]
        x = tf.keras.layers.IntegerLookup(vocabulary=cat_vocab[k], oov_token=0)(x)
        x = tf.keras.layers.Embedding(len(cat_vocab[k]) + 1, hidden_size)(x)
        embeddings[k] = x
    
    x = tf.concat(embeddings.values(), axis = -2)
    x = tf.keras.layers.Layer(name = 'embeddings')(x)
    return x

hidden_size = 16
embeddings = build_encoder(inputs, hidden_size)

context = tf.math.reduce_mean(embeddings, axis = 0, keepdims=True)

x = embeddings
x = BatchNormTransformerEncoder(hidden_size, 4, hidden_size, 0.1)(x)
embeddings = FieldVariableSelectionWithContext(x.shape[1], 4 * hidden_size, 0.1)(x, context)

team_id = tf.keras.layers.Input((1,), dtype=tf.int32, name='team_id')
teammate_matrix = tf.cast(team_id == tf.transpose(team_id), tf.float32)

users_team = teammate_matrix @ embeddings / tf.reshape(tf.reduce_mean(teammate_matrix, axis = 1), [-1,1])
x = tf.concat([embeddings, users_team], axis = 1)
# x = tf.math.softmax(users_team) * embeddings
x = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1),
    ], 
    name = 'output',
)(x)
x = tf.math.sigmoid(x)
model = tf.keras.Model(list(inputs.values()) + [team_id], x)

encoder = tf.keras.Model(inputs = inputs, outputs = embeddings)

In [2]:
def feature_process(x):
    x = {'{}'.format(kv.split('\x03')[0]) : kv.split('\x03')[1].split('\x04') for kv in x.split('\x02')}
    for k in x.keys():
        if k in ['r_weapon1', 'r_weapon2', 'r_character_id']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1] * 5)[0:5]
        elif k in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan']:
            x[k] = ([int(i) for i in x[k] if i!= 'null'] + [-1])[0:1]
        else:
            x[k] = ([float(i) for i in x[k] if i[0].isdigit()] + [-1] * 5)[0:5]
    return x

def to_record(raw : pd.DataFrame, file_name : str):
    groups = raw.groupby('battle_id')
    with tf.io.TFRecordWriter(file_name, options = tf.io.TFRecordOptions(compression_type='GZIP')) as writer:
        for batch in tqdm(groups.groups):
            batch = groups.get_group(batch)
            
            example = tf.train.Example()
            feature = example.features.feature
            for name in ['battle_id', 'team_id', 'app_version', 'is_battle_number_lift', 'is_damage_amount_lift', 'is_ka_number_lift', 'is_survival_time_lift', 'is_retention']:
                feature[name].CopyFrom(
                    tf.train.Feature(int64_list = tf.train.Int64List(value = batch[name].to_list()))
                )
                
            for name, spec in inputs.items():
                if spec.dtype == tf.float32:
                    feature[name].CopyFrom(
                        tf.train.Feature(float_list = tf.train.FloatList(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
                else:
                    feature[name].CopyFrom(
                        tf.train.Feature(int64_list = tf.train.Int64List(
                            value = [x for xs in batch.feature for x in xs[name]]    
                        ))
                    )
            writer.write(example.SerializeToString())

query = """
    WITH base AS (
        SELECT *
        FROM hive.ml_tmp_db.solarland_battle_sample_240129
        WHERE dt = '{ds}' 
    ) 

    SELECT
        a.battle_id
        , a.open_id
        , a.team_id
        , a.feature
        , a.is_battle_number_lift
        , a.is_retention
        , a.is_damage_amount_lift
        , a.is_ka_number_lift
        , a.is_survival_time_lift
    FROM base AS a
    INNER JOIN (
        SELECT battle_id
        FROM base
        WHERE battle_id is not null
        GROUP BY battle_id
        HAVING 
            COUNT(1) >= 2
            AND COUNT_IF(team_id is null) = 0
            AND MIN(is_retention) = 0 
            AND MAX(is_retention) = 1
    ) AS b
    ON a.battle_id = b.battle_id
    -- ORDER BY battle_id
    -- LIMIT 1000
"""

client = SQLClient('ethancao')
now = pendulum.date(2024, 1, 30)

dss = []

for delta in trange(2, 12):
    ds = now.add(days = - delta).to_date_string()
    dss.append(ds)
    
    #### continue if do not need to fetch dataset
    continue
    client.reset()
    client.execute_trino_query(query.format(ds = ds))
    raw = client.to_pandas()
    raw = raw.assign(
        battle_id = lambda x: x.battle_id.apply(CityHash32),
        open_id = lambda x: x.open_id.fillna('').apply(CityHash32),
        team_id = lambda x: x.team_id.astype(int),
        feature = lambda x: x.feature.apply(feature_process),
        app_version = lambda x: x.app_version.apply(lambda x: int(x[0]))
    )
    to_record(raw, file_name = f'../../data/tfrecord/solarland/{ds}_multitask.tfrecord')
    del raw
    # break
    # raw.to_parquet(f'../../data/raw/solarland/{ds}.parquet', compression='gzip', index = False)

100%|██████████| 10/10 [00:00<00:00, 9993.58it/s]


In [4]:
other_things = ['battle_id', 'team_id', 'app_version', 'battle_fight_num', 'is_battle_number_lift', 'is_damage_amount_lift', 'is_ka_number_lift', 'is_survival_time_lift', 'is_retention']

record_schema = {
    **{
        name : tf.io.VarLenFeature(dtype = spec.dtype if spec.dtype == tf.float32 else tf.int64)
        for name, spec in inputs.items()
    },
    **{
        name : tf.io.VarLenFeature(dtype = tf.int64)
        for name in other_things
    }
}

def decode_fn(msg):
    example = tf.io.parse_single_example(msg, record_schema)
    for name in record_schema:
        x = tf.sparse.to_dense(example[name])
        
        if name not in ['r_elo', 'r_his_battle_number_3d', 'r_lifespan'] + other_things:
            x = tf.reshape(x, [-1, 5])
        else:
            x = tf.reshape(x, [-1, 1])    
            
        example[name] = x    
    
    # name = 'is_retention'
    # name = 'is_survival_time_lift'
    name = 'is_battle_number_lift'
    label = example.pop(name)
    # label = tf.cast(label, tf.float32)
    return example, label

def filter_fn(x, y):
    flag = tf.reduce_any(y == 0) & tf.reduce_any(y == 1)
    return flag

train_dataset = tf.data.TFRecordDataset(
    [
        f'../../data/tfrecord/solarland/{ds}_mt.tfrecord'
        for ds in dss[2:]    
    ], 
    compression_type='GZIP',
    num_parallel_reads=tf.data.AUTOTUNE,
).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(
    [
        f'../../data/tfrecord/solarland/{ds}_mt.tfrecord'
        for ds in dss[:2]    
    ], 
    compression_type='GZIP',
    num_parallel_reads=tf.data.AUTOTUNE,
).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

gen0 = train_dataset.as_numpy_iterator()
gen1 = valid_dataset.as_numpy_iterator()

In [5]:
lr = tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 5000, 0.96)
# lr = tf.keras.optimizers.schedules.CosineDecay(1e-3, 1000, alpha = 0.1)

model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate = lr),
    # loss = tf.keras.losses.BinaryFocalCrossentropy(),
    loss = tf.keras.losses.BinaryCrossentropy(),
    # loss = tf.nn.log_poisson_loss,
    metrics = [tf.keras.metrics.AUC(name = 'auc'), tf.keras.metrics.BinaryAccuracy(name = 'acc')]
)
model.fit(
    gen0, 
    validation_data = gen1, 
    steps_per_epoch=10000, 
    validation_steps=4000,
    epochs = 100,
    callbacks=[
        # tf.keras.callbacks.TensorBoard('../../savedmodel/tensorboard/farlight84_attn'),
        tf.keras.callbacks.EarlyStopping('val_auc', patience = 10, min_delta = 0.01),
    ]   
)

Epoch 1/100


d:\Anaconda\envs\tf210\lib\site-packages\keras\engine\functional.py:637: UserWarning: Input dict contained keys ['app_version', 'battle_fight_num', 'battle_id', 'is_damage_amount_lift', 'is_ka_number_lift', 'is_retention', 'is_survival_time_lift'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


10000/10000 [==============================] - 216s 21ms/step - loss: 0.2789 - auc: 0.6816 - acc: 0.9170 - val_loss: 1.0866 - val_auc: 0.5709 - val_acc: 0.5497
Epoch 2/100
10000/10000 [==============================] - 217s 22ms/step - loss: 0.2195 - auc: 0.8418 - acc: 0.9176 - val_loss: 1.1732 - val_auc: 0.5744 - val_acc: 0.5471
Epoch 3/100
10000/10000 [==============================] - 212s 21ms/step - loss: 0.1842 - auc: 0.9101 - acc: 0.9156 - val_loss: 0.9621 - val_auc: 0.5918 - val_acc: 0.5544
Epoch 4/100
10000/10000 [==============================] - 214s 21ms/step - loss: 0.1750 - auc: 0.9222 - acc: 0.9142 - val_loss: 0.8457 - val_auc: 0.6070 - val_acc: 0.5609
Epoch 5/100
10000/10000 [==============================] - 208s 21ms/step - loss: 0.1670 - auc: 0.9277 - acc: 0.9177 - val_loss: 0.7816 - val_auc: 0.6163 - val_acc: 0.5703
Epoch 6/100
10000/10000 [==============================] - 209s 21ms/step - loss: 0.1623 - auc: 0.9322 - acc: 0.9183 - val_loss: 0.8761 - val_auc: 0.607

In [5]:
# model.evaluate(train_dataset.as_numpy_iterator())

In [6]:
# model.evaluate(valid_dataset.as_numpy_iterator())

In [7]:
# layer = model.get_layer('output')

# schema = {ipt.name : tf.TensorSpec(ipt.shape, ipt.dtype, name = ipt.name) for ipt in encoder.inputs}
# def graph_edge_score(batch):
#     user_cnt = tf.gather(tf.shape(batch['r_elo']), 0)
#     embeddings = encoder(batch)
#     matrix0 = tf.reshape(tf.repeat(tf.expand_dims(tf.eye(user_cnt), 0), repeats = user_cnt, axis = 0), [-1,user_cnt])
#     matrix1 = matrix0 + tf.repeat(tf.eye(user_cnt), repeats = [user_cnt], axis = 0) 
#     x = matrix0 @ embeddings
#     team = matrix1 @ embeddings / 2.0
#     x = tf.concat([x, team], axis = 1)
    
#     # square_of_sum = tf.square(tf.reduce_sum(x, axis = 1, keepdims=True))
#     # sum_of_square = tf.reduce_sum(tf.square(x), axis = 1, keepdims=True)
#     # cross = square_of_sum - sum_of_square
#     # cross = 0.5 * tf.reduce_sum(cross, axis = 1, keepdims=True)
    
#     x = tf.math.sigmoid(layer(x))
#     x = tf.reshape(x, [user_cnt, user_cnt])
#     return x


# model.save('test', signatures= {'serving_default' : tf.function(graph_edge_score, input_signature=[schema])})